# TP1 Transfert de chaleur en milieu poreux

__5 séances__

L'objectif est de développer un module python qui résout le transfert de chaleur en milieu poreux en régime transitoire thermique.


## 1. L'interface nappe-rivière en régime permanent

L'interface nappe-rivière est ici représentée par une portion de zone hyporhéique mono-dimensionnelle de hauteur h.

Le couple d'équations régissant le transfert de chaleur correspond d'une part à la résolution du régime permanent hydraulique (loi de Darcy):

$ \boldsymbol{q} = -K \boldsymbol{\nabla} H $

couplée à l'équation de transfert de chaleur

$ \rho_w c_w \boldsymbol{q} \cdot \boldsymbol{\nabla} \theta - \lambda_m \Delta \theta = 0 $


avec $\boldsymbol{q}$, le débit spécifique [$m.s^{-1}$],  
$H$, la charge [$m$]  
$K$, la perméabilité [$m.s^{-1}$],  
$\theta$, la température [$K$],  
$\rho_w$, la densité de l'eau [$kg.m^{-3}$],  
$c_w$ la capacité calorifique spécifique de l'eau [$J.kg^{-1}.K^{-1}$],  
$\lambda_m$ [$W.m^{-1}.K^{-1}$] la conductivité thermique du milieu poreux équivalent, avec $\lambda_m = \left( n \sqrt{\lambda_w} + (1-n) \sqrt{\lambda_s} \right)^2$ où $n$ est la porosité du milieu, et les indices $w$ et $s$ dénotent respectivement l'eau et le solide pur.

$\boldsymbol{\nabla}$ est un opérateur différenciel et $\Delta$ le Laplacien. 
Les symboles en gras représentent des vecteurs. 

Afin de rendre le programme générique permettant éventuelelment de prendre en compte les effets de température sur les propriétés de l'eau, la loi de Darcy est écrite dans sa version intrinsèque :

$ \boldsymbol{q} = - \frac{k \rho_w g}{\mu_w} \boldsymbol{\nabla} H $

avec where $k$ la perméabilité intrinsèque de l'eau [$m^2$],  
$g$ la constante gravitaire [$m.s^{-2}$],  
$\mu_w$ la viscosité dynamique de l'eau [$kg.s^{-1}$].


Le système expérimental MOLONARI (MOnitoring LOcal des échanges NAppe-RIvière) permet de mesurer la différence de pression entre le haut et le bas de la colonne.


### 1.1. Discrétiser l'équation en différences finies

Il s'agit ici de discrétiser l'équation du transfert de chaleur en régime permanent en différences finies.
Le problème est mono-dimensionnel vertical (une colonne de sol).

On considère un repère orienté vers le bas avec le 0 à la surface du sol.

Discrétiser les conditions limites de température qui sont imposées en haut et en bas de la colonne, soit aux faces.

On suppose que la colonne comprend n cellules, les conditions limites de température sont donc imposées aux faces 1/2 et n+1/2.





__Solution__ :

Le débit est constant tout au long de la colonne :

$ \boldsymbol{q} = -K_{eq} \frac{H_{top}-H_{bot}}{L} $,

L la longueur de la colonne de sol.


$ \rho_w c_w \boldsymbol{q} \cdot \boldsymbol{\nabla} \theta - \lambda_m \Delta \theta = 0 $

Soit $\kappa =  \frac{\lambda_m}{\rho_w c_w }$
alors $ \boldsymbol{q} \cdot \boldsymbol{\nabla} \theta - \kappa \Delta \theta = 0 $

__Discrétisation du premier terme__

$q_i \nabla \theta_i = q_i \frac{\theta_{i+1} - \theta{i-1}}{2 dz}$

dz longueur de la cellule,  

$q_i$ est la moyenne du débit aux faces: $q_i = \frac{q_{i+\frac{1}{2}}+ q_{i-\frac{1}{2}}}{2}$ 

Dans le cas permanent, ce débit est constant.

Si i est une cellule limite à température imposée alors i-1 devient i-1/2 et la formule :

$\nabla \theta_i = \frac{\theta_{i+1} - \theta{i-\frac{1}{2}}}{\frac{3}{2} dz}$

__Discrétisation du second terme__

$\Delta \theta_i = \nabla \nabla \theta_i$  
$\Delta \theta_i = \nabla \frac{\theta_{i+\frac{1}{2}} - \theta_{i-\frac{1}{2}}}{dz} $  
$\Delta \theta_i = \frac{\theta_{i+1} - 2 \theta_{i} + \theta{i-1}}{dz^{2}} $


Si i est une cellule limite à température imposée alors i-1 devient i-1/2  
$\Delta \theta_i = \nabla \frac{\theta_{i+\frac{1}{2}} - \theta_{i-\frac{1}{4}}}{\frac{3}{4}dz} $  
$\Delta \theta_i = \frac{4}{3 dz} [ \nabla \theta_{i+\frac{1}{2}} - \nabla \theta_{i-\frac{1}{4}} ] $   
$\Delta \theta_i = \frac{4}{3 dz} [ \frac{\theta_{i+1}-\theta_{i}}{dz} - 2 \frac{\theta_i - \theta_{i-\frac{1}{2}}}{dz} ] $  
$\Delta \theta_i = \frac{4}{3 dz^{2}} [ \theta_{i+1}-3 \theta_{i} + 2 \theta_{i-\frac{1}{2}} ] $  

__On obtient ainsi pour une cellule quelconque :__


$\theta_{i+1} - \theta_{i-1} - \frac{2 \kappa_i}{q_i dz} (\theta_{i+1} - 2 \theta_{i} + \theta_{i-1}) = 0$

$\boldsymbol{(q_i  - \frac{2 \kappa_i}{dz}) \theta_{i+1} + \frac{4 \kappa_i}{dz} \theta_{i}  - (q_i  +  \frac{2 \kappa_i}{dz}) \theta_{i-1} = 0}$

__Et pour une cellule limite à température imposée__  


$q_i (\frac{\theta_{i+1} - \theta{i-\frac{1}{2}}}{\frac{3}{2} dz}) - \kappa_i \frac{4}{3 dz^{2}} (\theta_{i+1}-3 \theta_{i} + 2 \theta_{i-\frac{1}{2}}) = 0$


$\theta_{i+1} - \theta_{i-\frac{1}{2}} - \frac{2 \kappa_i}{q_i  dz} ( \theta_{i+1}-3 \theta_{i} + 2 \theta_{i-\frac{1}{2}} ) = 0$


$\theta_{i+1} - \frac{2 \kappa_i}{q_i  dz} ( \theta_{i+1}-3 \theta_{i} ) = \theta_{i-\frac{1}{2}} + \frac{4 \kappa_i}{q_i  dz} + \theta_{i-\frac{1}{2}} $


$\boldsymbol{(q_i   - \frac{2 \kappa_i}{ dz} ) ( \theta_{i+1}) + \frac{6 \kappa_i}{ dz} \theta_{i}  = (q_i  + \frac{4 \kappa_i}{dz} )\theta_{i-\frac{1}{2}}} $

### 1.2. Formalisation du maillage, des variables d'état et des paramètres

En python, formaliser le maillage, identifier les variables d'état, les paramètres et les conditions limites.
Ecrire le code nécessaire à la lecture de ces données.

On considère une colonne de sol de 1 m d'épaisseur. La discrétisation mobilise des cellules de 1cm de côté.

La différence de charge entre le haut et le bas de la colonne est donnée par le système MOLONARI. Elle vaut 5cm.

Les propriétés de la colonne de sol sont dans un premier temps :

"permeability" : {  
            "val": "1e-5",  
            "unit": "m/s"  
        },  
        "porosity" : {  
            "val" : "0.15"  
        }  

"sediment": {  
        "specificHeatCapacity" : {  
            "val": "957",  
            "unit" : "m2 s-2 K-1"  
        },  
        "lambda" : {  
            "val": "2",   
            "unit": "W m-1 K-1"  
        },  
        "rho" : {  
            "val" : "2600",  
            "unit" : "kg/m3"  
        }  
    }  

"Triv" : {  
        "val" : 29,  
        "unit" : "°C"  
    },  
"Taq" : {  
        "val" : 12,  
        "unit" : "°C"  
    }  


#### 1.2.1 Les classes considérées

La classe principale est la classe Column. Elle mobilise de nombreuses autres classes :

+ Point dans geometry.py 
+ Geometry dans geometry.py 
+ Face  dans geometry.py 
+ Cell  dans geometry.py 
+ PropPorousMedia  dans porousMedia.py 
+ PropHydro dans hydrogeology.py 
+ PropMedia dans heat.py 
+ Hydro dans hydrogeology.py 
+ Heat dans heat.py 
+ BoundaryConditionHyd  dans hydrogeology.py 
+ BoundaryConditionHeat dans heat.py 
+ LinSys dans linearAlgebra.py 

L'ensemble des fichiers *.py sont regroupés pour former un package dans le répertoire codepyheat. Ce package est initialisé par __init__.py qui permet de définir les paths relatifs, les CONST et les functions pour les imprimer en str. Le main.py peut ainsi être exécuté. Afin de rentre l'exécution du main encore plus générique, et surtout pour faire tourner les *.py de codepyheat indépendemment, il faut définir une variable globale d'environnement, en ajoutant le chemin vers le projet (ie le parent de codepyheat) à PYTHONPATH.

Enfin le fichier units.py assure les convertions d'unités en ayant recours au fichier de configuration __convOpTable.json qui ne doit pas être modifié par les utilisateurs.

Afin de pouvoir faire tourner le code dans le jupyter notebook, il est nécessaire de tout d'abord définir une metaclasse utile à la lecture des fichiers. Nous reviendrons donc ultérieurement sur le code relatif aux classes.

#### 1.2.2 Instanciation des objets

L'instanciation des objet est basé sur la méthode __init__() de la classe correspondante. Par défaut, pour les classes qui nécessitent une définition par l'utilisateur, cette initialisation est basée sur l'utilisation d'un dictionary (dict). Afin de permettre une configuration aisée du code via des fichiers txt, il est possible de réaliser l'instanciation par le biais d'un fichier JSON. La définition de la méthode de classe d'instanciation par lecture de fichier est écrite dans factory.py : FactoryClass. Elle utilise des décorateurs et requière de ses héritiers de spécifier obligatoirement une méthode __init__ qui pourra être decorée par les méthodes méthodes de FactoryClass. Les classes dont les instances requiert une lecture de fichiers héritent de FactoryClass.  


#### 1.2.3 La Classe Column

Elle contient l'ensemble des autres Classes et des methodes pour effectuer le calcul direct.
Auparavant définir PYTHONPATH here:

Les fichiers de configuration json sont tous contenus dans le répertoire json de codepyheat

Le premier configColumn.json décrit la géométrie de la colonne :  
{  
    "depth": {  
            "val": "100",  
            "unit": "cm"  
        },  
    "ncells": 100    
}  

Le second paramColumn.json décrit les noms des fichiers décrivant les propriétés de la colonne :  
{  
    "name": "Soil Column",  
    "hydroFile": "paramHyd.json",  
    "sedFile": "paramSed.json"    
}  

paramHyd.json décrit les propriétés hydrodynamiques :  
{  
    "hydro": {   
        "permeability" : {  
            "val": "1e-5",  
            "unit": "m/s"  
        },  
        "porosity" : {  
            "val" : "0.15"  
        }  
    }  
}  

paramSed.json, les propriétés thermiques des grains du milieu poreux:  
{    
    "sediment": {  
        "specificHeatCapacity" : {  
            "val": "957",  
            "unit" : "m2 s-2 K-1"  
        },  
        "lambda" : {  
            "val": "2",   
            "unit": "W m-1 K-1"  
        },  
        "rho" : {  
            "val" : "2600",  
            "unit" : "kg/m3"  
        }  
    }  
}  

Les conditions limites hydrauliques et thermiques sont contenues respectivement dans :
+ configBcHydro :  
{  
    "dH" : {  
        "val": 5,  
        "unit": "cm"  
    }  
}   

+ et configBcTemp :  
{  
    "Triv" : {  
        "val" : 29,  
        "unit" : "°C"  
    },  
    "Taq" : {  
        "val" : 12,  
        "unit" : "°C"  
    }  
}  




### 1.4. Sorties du code

Ecrire les champs de températures dans un fichier.
Ecrire une procédure de visualisation des champs de température. On pourra se baser ici sur matplotlib



Les méthodes d'impression et de visualisation sont définies dans geometry.py. Il est possible de les lancer via un script python basé sur le package codepyheat.

### 1.5. Ecrire une fonctionnalité du code permettant l'analyse de sensibilité

Il s'agit d'écrire une méthode permettant de lancer un calcul direct en spécifiant les valeurs des trois paramètres intervenant dans le système d'équations : 
+ la perméabilité 
+ la conductivité thermique 
+ la porosité cinématique




## 2. Régime thermique transitoire


### 2.1 Forme générale


On considère la convention de notation suivante. $\boldsymbol{\nabla}$ est un opérateur différenciel et $\Delta$ le Laplacien. 
Les symboles en gras représentent des vecteurs. 

L'interface nappe-rivière est ici représentée par une portion de zone hyporhéique mono-dimensionnelle de hauteur h.

Le couple d'équations régissant le transfert de chaleur correspond d'une part à la résolution du régime permanent hydraulique (loi de Darcy):

$ \boldsymbol{q} = -K \boldsymbol{\nabla} H $

avec $\boldsymbol{q}$, le débit spécifique [$m.s^{-1}$],  
$H$, la charge [$m$]  
$K$, la perméabilité [$m.s^{-1}$],

Le système expérimental MOLONARI (MOnitoring LOcal des échanges NAppe-RIvière) permet de mesurer la différence de pression entre le haut et le bas de la colonne, qui est ainsi une donnée d'entrée du problème. A perméabilité considérée homogène le long de la colonne, le débit est ainsi évaluable pour le couplage avec l'équation de transfert de chaleur en régime permanent :

$\rho_m c_m \dfrac{\partial \theta}{\partial t} = - \rho_w c_w \boldsymbol{q} \cdot \boldsymbol{\nabla} \theta + \lambda_m \Delta \theta$

avec $\theta$, la température [$K$],  
$\rho_i$, la densité de i [$kg.m^{-3}$],  
$c_i$ la capacité calorifique spécifique de i [$J.kg^{-1}.K^{-1}$],  
$\lambda_i$ [$W.m^{-1}.K^{-1}$] la conductivité thermique de i,
où $i \in {s,w,m}$, avec s solide, w eau, m milieu poreux equivalent. 

$c_m\rho_m$ est estimé à partir de moyenne volumique pondérée par la porosité $n$ du milieu, $ \rho_m c_m = n \rho_w c_w + (1 - n) \rho_s c_s $

La conductivité thermique du milieu poreux équivalent est estimée par la relation $\lambda_m = \left( n \sqrt{\lambda_w} + (1-n) \sqrt{\lambda_s} \right)^2$.

### 2.2 Réduction de l'espace des paramètres

L'équation de transport de la chaleur peut être réécrite sous la forme :


$ \dfrac{\partial \theta}{\partial t} = \kappa_e \Delta \theta + 
\alpha_e \boldsymbol{\nabla} H \cdot \boldsymbol{\nabla} \theta $

avec $ \kappa_e = \dfrac{\lambda_m}{ \rho_m c_m }  $

$ \alpha_e = \dfrac{\rho_w c_w}{ \rho_m c_m} K $


$\kappa_e$ [$m^2 s^{-1}$] et $\alpha_e$ [$m s^{-1}$] sont des paramètres effectifs, dénommés respectivement conductivité effective (parfois dénommée effective diffusivité thermique) et paramètre advectif effectif.

### 2.3 Propagation d'un signal périodique dans un milieu semi infini

Il existe une solution analytique, fournie par Stallman (1965), à la propagation d'un signal de température sinusoïdal  à la surface d'un milieu poreux soumis à une différence de pressions ($\Delta H$) constante au cours du temps. Le signal sinusoïdal est apliqué en haut de la colonne, dans un repère dont l'origine est en haut de la colonne et orienté vers le bas. Il a une amplitude  $\theta_{amp}$ et une période P autour d'une valeur moyenne $\theta_{\mu}$. Cette solution s'écrit :

$ \theta(z,t)  = \theta_{\mu} + \theta_{amp} e^{-az} \cos\left( \dfrac{2 \pi}{P} t - b z \right) $  [eq.1]

avec 
$ a  = \dfrac{1}{2 \kappa_e} \left( \sqrt{\dfrac{\sqrt{v_t^4 + (8 \pi \kappa_e / P)^2 } + v_t^2}{2}} - v_t \right) $, 

$ b  = \dfrac{1}{2 \kappa_e}  \sqrt{\dfrac{\sqrt{v_t^4 + (8 \pi \kappa_e / P)^2 } - v_t^2}{2}} $, 

$ v_t  = - \alpha_e \dfrac{\partial H}{\partial z} $


Dans le cas conductif pur, la solution analytique s'écrit:

$ \theta(z,t)  = \theta_{\mu} + \theta_{amp} e^{-\sqrt{\dfrac{\pi}{\kappa_e P}}z} \cos\left( \dfrac{2 \pi}{P} t - \sqrt{\dfrac{\pi}{\kappa_e P}} z \right) $  [eq.1cond]



### 2.4 Etude de cas

Considérons une colonne de sol de 8 m de profondeur avec la conductivité thermique équivalente $\lambda_m = 1 W.m^{-1}.K^{-1}$, la porosité $ n = 0.15$, et la capacité calorifique équivalente $\rho_m c_m =  4e6 J.m^{-3}.K^{-1}$. La perméabilité est fixée à $8.10^{-4} m.s^{-1}$. 

#### 2.4.1 Réponse de la zone hyporhéique à une sollicitation thermique périodique

On considère un signal périodique en surface d'amplitude 1 K et une période de 720 heures (30 jours) autour d'un température moyenne de 12 °C.

##### 2.4.1.1 Evolution de la température dans le milieu

Tracer le profil des températures toutes les 30 heures pour les trois cas suivants :

+ 1. vitesse de darcy vers le haut de 1e-6 $m. s^{-1}$
+ 2. profil conductif pur
+ 3.  vitesse de darcy vers le bas de 1e-6 $m. s^{-1}$

Avant de démarrer, écrire une méthode permettant de forcer une simulation à partir de paramètres thermiques équivalents (conductivité thermique et capacité calorifique).

Tracer également des frises temporelles représentant la matrice des températures dans l'espace (z,t). 

Commentez ces résultats.

##### 2.4.1.2 Quantifier et représentez les flux d'énergie relatifs à ces états thermiques

Tracer également des frises temporelles représentant les flux de chaleur advectifs et conductifs relatifs aux trois cas précédents. Commentez ces résultats.

##### 2.4.1.3 Profondeur de pénétration

Par convention on considère que la profondeur de pénétration d'une onde est définie par la profondeur à partir de laquelle l'amplitude est amortie de $e^{-1}$, soit environ d'un tiers.

__Calculer la profondeur de pénétration__

__Tracer les profondeurs de pénétration__ pour des signaux périodiques dont la période varie entre 1hr et 1 an. Pour des vitesses descendantes, ascendantes variant en valeur absolue entre 10$^{-5}$ et 10$^{-8}$ m.s$^{-1}$. On utilisera des échelles log.

Commentez ces résultats.

##### 2.4.1.4 Temps d'arrivée d'une perturbation à une profondeur donnée

En analysant la composante ondulatoire de a solution analytique [eq.1], définir le __temps d'arrivée__ à une profondeur $z$ d'une perturbation de période $P$. En déterminer la vitesse la vitesse de propagation de l'onde.

__Tracer les vitesses de propagation__ pour une gamme de vitesses variant entre 1e-5 m/s et 1e-8 m/s, et pour le cas conductif. Commentez vos résultats

#### 2.4.2 Sollicitations multi-périodiques
 
On considère maintenant deux signaux périodiques emboîtés, l'un annuel (d'amplitude 6 °C), l'autre journalier d'amplitude 1°C.

Représenter tout d'abord les deux influences séparemment puis voir comment elles se combinent dans le milieu pour les deux cas infliltrant et exfiltrant à une vitesse de Darcy de +/-1e-6 $m. s^{-1}$. Représentez les profils verticaux et les frises.

Commentez les résultats obtenus.

## 3. Modèle direct hydro-thermique

Intégrer la résolution transitoire de l'équation de la diffusivité à votre code afin d'obtenir un code capable de résoudre le problème hydro-thermique complet.